In [ ]:
from __init__ import *

In [ ]:
comparison_x_sample = pd.read_csv(
    "../output/comparison_x_sample.tsv", sep="\t", index_col=0
)

set_x_sample = pd.read_csv("../output/set_x_sample.tsv", sep="\t", index_col=0)

In [ ]:
name_to_set_to_features = {}

directory_path = "../input/set/"

for name in os.listdir(path=directory_path):

    name_to_set_to_features[name] = kraft.gmt.read("{}/{}".format(directory_path, name))

In [ ]:
keyword_arguments = {
    "se_data_type": "binary",
    "plot_std": SETTING["plot_std"],
}

for comparison, groups in comparison_x_sample.iterrows():

    groups.dropna(inplace=True)

    min_group_size = np.unique(groups.to_numpy(), return_counts=True)[1].min()

    for function, group_size in ((kraft.information.get_ic, 2),):

        if min_group_size < group_size:

            continue

        directory_path = "../output/compare_set/{}/{}/".format(
            comparison, function.__name__
        )

        kraft.path.path(directory_path)

        scores = kraft.function_heat_map.make(
            groups,
            set_x_sample,
            function,
            n_job=SETTING["n_job"],
            n_sampling=SETTING["n_sampling"],
            n_permutation=SETTING["n_permutation"],
            n_extreme=SETTING["n_extreme"],
            title="All ({})".format(function.__name__),
            directory_path=directory_path,
            **keyword_arguments,
        )

        if 0 < len(SETTING["sets_to_peek"]):

            kraft.function_heat_map.make(
                groups,
                set_x_sample.reindex(labels=SETTING["sets_to_peek"]),
                scores,
                n_extreme=None,
                title="Peek ({})".format(function.__name__),
                **keyword_arguments,
            )

        for name, sets in name_to_set_to_features.items():

            directory_path_ = "{}/{}/".format(directory_path, name)

            kraft.path.path(directory_path_)

            common_sets = set_x_sample.index & sets

            if 0 < len(common_sets):

                kraft.function_heat_map.make(
                    groups,
                    set_x_sample.loc[common_sets, :],
                    scores,
                    n_extreme=SETTING["n_extreme"],
                    title="{} ({})".format(name, function.__name__),
                    directory_path=directory_path_,
                    **keyword_arguments,
                )